In [4]:
# http://127.0.0.1:5000/v1/ 

from flask import Flask, request, jsonify
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from retriever_module.retriever import llm_wrapper
app = Flask(__name__)

# # 加载模型和分词器
# llm = llm_wrapper(model_path_or_name="/home/simon/disk1/Simon/Code/LLM/models/meta-llama/Llama-2-7b-hf")

# llm("hi")

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from transformers import BitsAndBytesConfig



import torch 
class MultiConversation: 
    def __init__(self, model_name, init_prompt, device='cuda' if torch.cuda.is_available() else 'cpu'): 
        nf4_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
        )
        self.device = device 
        self.tokenizer =  AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config = nf4_config)
        self.messages = [] 
        self.messages.append(self.tokenizer(init_prompt, return_tensors="pt"))
    def qa(self, question): 
        # 将新问题添加到消息列表中 
        question_tensors = self.tokenizer(question, return_tensors="pt")
        # 将新问题和历史消息合并（注意：这里需要稍微处理一下，因为模型可能需要特定的输入格式） 
        # # 假设模型可以直接接受这样的序列（这取决于模型的实际要求） 
        # # 注意：这里只是简单地将它们连接在一起，实际情况可能需要更复杂的处理 
        # print(self.messages)
        # print([m['input_ids'] for m in self.messages])
        print([m['input_ids'] for m in self.messages] + [question_tensors['input_ids']])
        combined_input = torch.cat([m['input_ids'] for m in self.messages] + [question_tensors['input_ids']], dim=1) 
        attention_mask = torch.cat([torch.ones_like(m['attention_mask']) for m in self.messages] + [question_tensors['attention_mask']], dim=1) 
        # 你可以根据需要调整这些参数（如 max_length, top_p, temperature 等） 
        generated_ids = self.model.generate( input_ids=combined_input, attention_mask=attention_mask, max_length=100, 
                                            # 你可以根据需要调整这个长度 
                                            temperature=1.0, top_p=0.9, top_k=0, repetition_penalty=1.0, do_sample=True, ) 
        # 解码生成的 ID 以获取文本 
        answer = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True) 
        #保存回答到消息列表中（这里只是简单记录，实际使用可能需要更复杂的数据结构
        self.messages.append({ 'input_ids': question_tensors['input_ids'], 'attention_mask': question_tensors['attention_mask'], 'generated_text': answer }) 
        return answer 
        

In [20]:
#使用示例 
model_name = '/home/simon/disk1/Simon/Code/LLM/models/nvidia/Llama3-ChatQA-1.5-8B' 
init_prompt = "Hello, how are you?" 
conv = MultiConversation(model_name, init_prompt) 
print(conv.qa("I'm fine. How about you?"))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.11s/it]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/simon/miniconda3/envs/LLM/lib/python3.10/site-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


[tensor([[9906,   11, 1268,  527,  499,   30]]), tensor([[  40, 2846, 7060,   13, 2650,  922,  499,   30]])]
Hello, how are you?I'm fine. How about you?I'm good, thanks for asking. How's the new job?It's going well. I like it a lot.


In [ ]:
# # 实现一个多轮问答的类
# class MultiConversation:
#     # 初始化对象时传入第一句指令
#     def __init__(self, init_prompt):
#         self.messages = []
#         self.messages.append({"role": "system", "content": init_prompt})

#     # 每调用一次这个方法，都会将问题和回答记录在self.messages列表属性中，用于记录上下文
#     def qa(self, question):
#         self.messages.append({"role": "user", "content": question})
#         answer = 
#         self.messages.append({"role": "assistant", "content": answer.choices[0].message.content})
#         return answer.choices[0].message.content

In [21]:
model_name = '/home/simon/disk1/Simon/Code/LLM/models/nvidia/Llama3-ChatQA-1.5-8B' 
init_prompt = "you are a userful helper" 
conv = MultiConversation(model_name, init_prompt) 
@app.route('/v1/chat/completions', methods=['POST'])
def predict():
    # 解析请求数据
    data = request.json
    text = data['text']
    
    ans = conv.qa(text)
    return jsonify({'content': ans})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.08s/it]


In [10]:
if __name__ == '__main__':
    app.run(debug=True,port=5000)

In [9]:
!kill -9 3256561

In [8]:
!netstat -nplt

（并非所有进程都能被检测到，所有非本用户的进程信息将不会显示，如果想看到所有信息，则必须切换到 root 用户）
激活Internet连接 (仅服务器)
Proto Recv-Q Send-Q Local Address           Foreign Address         State       PID/Program name    
tcp        0      0 0.0.0.0:80              0.0.0.0:*               LISTEN      -                   
tcp        0      0 0.0.0.0:443             0.0.0.0:*               LISTEN      -                   
tcp        0      0 0.0.0.0:1200            0.0.0.0:*               LISTEN      -                   
tcp        0      0 0.0.0.0:5455            0.0.0.0:*               LISTEN      -                   
tcp        0      0 0.0.0.0:9000            0.0.0.0:*               LISTEN      -                   
tcp        0      0 0.0.0.0:9001            0.0.0.0:*               LISTEN      -                   
tcp        0      0 0.0.0.0:9380            0.0.0.0:*               LISTEN      -                   
tcp        0      0 127.0.0.53:53           0.0.0.0:*               LISTEN      -                   
tcp        0  

In [3]:
!lsof -i:9059

COMMAND     PID  USER   FD   TYPE    DEVICE SIZE/OFF NODE NAME
python  1709195 simon   10u  IPv4 183131328      0t0  TCP localhost:9059 (LISTEN)
python  1709195 simon   17u  IPv4 182690398      0t0  TCP localhost:32808->localhost:9059 (ESTABLISHED)
python  1709195 simon   18u  IPv4 182690399      0t0  TCP localhost:9059->localhost:32808 (ESTABLISHED)
python  1709195 simon   60u  IPv4 183131346      0t0  TCP localhost:9059->localhost:32822 (ESTABLISHED)
code    3255260 simon   81u  IPv4 182690403      0t0  TCP localhost:32822->localhost:9059 (ESTABLISHED)
